Particle based simulation tutorial
==================

**Import LavaVu and create a Viewer object**

In [ ]:
import lavavu
lv = lavavu.Viewer()

**Create an initial set of random points and velocities**

In [ ]:
import random
import math

count = 1000 #Number of particles
positions = []
velocity = []

for p in range(0,count):
    #Random positions, with height offset of 3 added to y component
    positions.append([random.uniform(-0.05,0.05), 3 + random.uniform(-0.05,.05), random.uniform(-0.05,0.05)])

    #Random velocities for the points
    velocity.append([random.gauss(0,0.04), random.gauss(0,0.04), random.gauss(0,0.04)])


**Apply some global settings**

In [ ]:
lv["gpucache"] = True        #Enable caching all data on GPU for speed (don't use for very large datasets)
lv["background"] = "#eeeeee" #Set the plot background colour
lv["axis"] = False
lv["border"] = False

#Here we can set a fixed bounding box
#Without this the bounding box is elastic and calculated to fit the geometry
#lv["min"] = [-1.0, -1.0, -1.0]
#lv["max"] = [1.0, 1.0, 1.0]

**Plot ground plane and axis lines as an example of some static data (not time-varying)**

This data remains in the visualisation regardless of the time step and must be loaded first, before the time varying data.

In [ ]:
lines = lv.lines(colours="red green blue")
lines.vertices([[-1.0, 0, 0], [1.0, 0, 0], [0, -1.0, 0], [0, 1.0, 0], [0, 0, -1.0], [0, 0, 1.0]])

ground = lv.quads("ground", colour="darkgrey", cullface=False, dims=[2,2]);
gplane = [[-5, -5], [5, 5]]
ground.vertices([[gplane[0][0], 0.0, gplane[0][1]], 
                 [gplane[1][0], 0.0, gplane[0][1]],
                 [gplane[0][0], 0.0, gplane[1][1]],
                 [gplane[1][0], 0.0, gplane[1][1]]])

**Setup the particle drawing object**

In [ ]:
#Plot time varying data: points
points = lv.points(pointsize=10, pointtype="shiny", opacity=0.75)

#Apply a colourmap, cubeHelix() generates a map with monotonically increasing luminance
cmap = points.colourmap(lavavu.cubeHelix()) #, range=[0,0.1])

#Add a colour bar
points.colourbar()


**Advect the random particles and reload the positions for each time step**

In [ ]:
#This is a very simple toy particle simulation for the sake of demonstration only
steps = 300 #Number of steps to run for
for s in range(steps):
    values = []
    #Loop through particles
    for i in range(len(positions)):
        p = positions[i]
        v = velocity[i]
        
        for c in range(3):
            #Advect
            p[c] = p[c] + v[c]
            
            #Apply drag
            v[c] *= 0.99

        #Bounce off floor
        if p[1] < 0:
            p[1] = 0
            v[1] = -0.9*v[1]
                
        #Gravity
        v[1] -= 0.001

        #Values for the points: velocity magnitude
        #these values will be used to colour the points
        values.append(math.sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2]))

    #Add a new time step
    lv.addstep()

    #Load updated positions and values
    points.vertices(positions)
    points.values(values)


**Get the timestep information**

In [ ]:
lv.timestep(0)
steps = lv.timesteps()
print len(steps)
print steps[0:10]

**Filters allow elements to be filtered based on their data sets**

In [ ]:
#Clear filters
points["filters"] = []

#Filter to a range of values
myfilter = points.includemap('default', (0.0, 1.0))

**Build a control panel to view and modify the visualisation**

In [ ]:
lv.control.Panel()
lv.control.TimeStepper()
points.control.Range("pointsize", range=(1.,20), step=0.5, command="reload")
points.control.Filter(myfilter)
lv.control.ObjectList()
lv.control.show()

**Get the current camera state**

This outputs the commands necessary to restore the camera to its current settings, which can be copied and pasted to save a viewpoint for re-use

In [ ]:
lv.camera()

**Create a video animation**

Before generating a video we change adjust the visualistion in the interactive view, or use a previously saved camera to set a nicer viewpoint as follows:

In [ ]:
lv.translation(-0.0697407573461533, 0.569142580032349, -14.53618717193)
lv.rotation(0.140467837452888, 0.303975164890289, 0.0171317551285028, 0.94210988283157)

In [ ]:
lv.video(resolution=(500,400), fps=50)